# Maximisation de la fonction de vraissemblance 

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from scipy.optimize import minimize

## Quelques notations 

- $X \equiv$ un vecteur contenant les déterminants observables de $T_s$ et $T_d$ (genre, jour de naissance, département de naissance, etc.)

- $\tau_{birth} \equiv$ la date de naissance de l'individu

- $\tau_{contract} \equiv$ la date de la vente en viager

- $\tau_{begin} \equiv$ la date du début de la période d'observation des fichiers INSEE (1-1-1970)

- $\tau_{end} \equiv$ la date de la fin de la période d'observation des fichiers INSEE (1-6-2023)

- $V_d \equiv$ une variable captant l'effet des déterminants inobservables de $T_d$ (état de santé, richesse, niveau d'éducation)

- $V_s \equiv$ une variable captant l'effet des déterminants inobservables de $T_s$ (potentiellement les mêmes variables que celles captées par $V_d$, une variable indiquant si la personne a des enfants, une autre variable indiquant si elle a perdu son conjoint, etc.)

- $V \equiv (V_d,V_s)$

La distribution de $T_d$ conditionnellement à $T_s$, $X$ et $V$, notée $T_d|T_s,X,V$, est complètement caractérisée par le taux de hasard (comme $T_d$ représente une durée de vie il s'agit en fait d'un taux de mortalité) :
\begin{equation*}
\underset{\Delta \downarrow 0}{\text{lim}} \frac{Pr(t \leq T_d< t
+\Delta |T_d \geq t,T_s=t_s,X=x, V=v)}{\Delta} \equiv \theta_d(t|t_s,x,v).
\end{equation*}

Donc $\theta_d(t|t_s,x,v)\Delta$ s'interprète comme la probabilité qu'une personne décède durant l'interval $[t,t+{\Delta})$ sachant $T_s$, $X$ et $V$, et le fait que cette personne soit toujours vivante à l'âge $t$.

De la même manière, la distribution de $T_s$ conditionnellement à $X$ et $V$ est complètement déterminée par le taux de hasard
\begin{equation*}
\underset{\Delta \downarrow 0}{\text{lim}} \frac{Pr(t \leq T_s< t
+\Delta |T_s \geq t,X=x, V=v)}{\Delta} \equiv \theta_s(t|x,v).
\end{equation*}

Finalement, on note $F_d(t|t_s,x,v)=Pr(T_d<t|T_s=t_s,X=x, V=v)$ la fonction de répartition conditionnelle de $T_d$, et $f_d(t|t_s,x,v)=\partial F_d(t|t_s,x,v)/\partial t$ la fonction de densité associée. Ces fonctions sont liées au taux de hasard $\theta_d(t|t_s,x,v)$ de la façon suivante :
\begin{equation*}
    \theta_d(t|t_s,x,v)=\frac{f_d(t|t_s,x,v)}{1-F_d(t|t_s,x,v)}
\end{equation*}
et
\begin{equation*}
1-F_d(t|t_s,x,v)=exp\left[-\int_0 ^t \theta_d(u|t_s,x,v) du \right].
\end{equation*}

Faisons maintenant l'hypothèse que les taux de hasard s'écrivent comme :

$$ \theta_d(t|t_s,x,v)=
    \lambda_d(t) \phi_d(x) v_d \text{ si } t \le t_s \\
  \theta_d(t|t_s,x,v)=
    \lambda_d(t) \phi_d(x) \delta(t,t_s,x) v_d \text{ si } t >t_s $$

et

$$ \theta_s(t|x,v)=\lambda_s(t) \phi_s(x) v_s.$$

- Les fonctions $\lambda_d(t)$ et $\lambda_s(t)$ (parfois appelés taux de hasard de base) mesurent comment les taux de hasard dépendent respectivement de la durée $T_d$ et $T_s$. 

- Les termes $\phi_d(x)$ et $\phi_x(x)$ mesurent l'effet des variables observés sur les taux de hasard. 

- La composante cruciale dans ce modèle est $\delta(t,t_s,x)$. Cette dernière fonction capte comment le taux de mortalité change après la vente en viager. Elle mesure donc comment les supplements d'argent reçus par le vendeur affecte sa durée de vie, et cet effet peut dépendre de l'âge $t$, l'âge $t_s$ auquel l'individu a vendu son bien immobilier en viager, et les caractéristiques $x$.

### 1er cas : Modèle le plus simple 

On suppose que $var(V_d)=var(V_s)=0$ et $\delta(t,t_s,x)=\delta$.

$$
\theta_d(t|t_s,x)= 
    \lambda_d \phi_d(x) \text{ si } t \le t_s\\
\theta_d(t|t_s,x)= 
    \lambda_d \phi_d(x) \delta \text{ si } t >t_s
$$

et

$$
\theta_s(t|x)=\lambda_s \phi_s(x).
$$

On a donc fait l'hypothèse que l'hétérogénéité inobservée est absente et que les durées $T_d|T_s,X$ et $T_s|X$ suivent des lois exponentielles (les taux de hasard de base $\lambda_d(t)$ et $\lambda_s(t)$ ne dépendent pas de $t$).

$L_i^{seller} \space = \space \displaystyle \frac{f_d(t_{id}^{seller}|t_{is}^{seller},x_i)f_s(t_{is}^{seller}|x_i)}{Pr(\tau_{begin}<\tau_{i,birth}+T_{id}^{seller}<\tau_{end}|X_i=x_i)} \\$ 
<br>
<br>
$= \displaystyle \frac{f_d(t_{id}^{seller}|t_{is}^{seller},x_i)f_s(t_{is}^{seller}|x_i)}{\int \left[F_d(\tau_{end}-\tau_{i,birth}|t_s,x_i)-F_d(\tau_{begin}-\tau_{i,birth}|t_s,x_i) \right] f_s(t_s|x_i)dt_s} \\$
<br>
<br>
$= \displaystyle \frac{\lambda_d \phi_d(x_i) \delta e^{-\phi_d(x_i)I_d(t_{id}^{seller},t_{is}^{seller})} \lambda_s \phi_s(x_i) e^{-\phi_s(x_i)I_s(t_{is}^{seller})}}{\int \left[e^{-\phi_d(x_i)I_d(\tau_{begin}-\tau_{i,birth},t_s)}-e^{-\phi_d(x_i)I_d(\tau_{end}-\tau_{i,birth},t_s)}\right] \lambda_s \phi_s(x_i) e^{-\phi_s(x_i)I_s(t_s)} dt_s}$
<br>
<br>
<br>
<br>

$L_i^{clone} = \displaystyle \int_{t_{id}^{clone}} ^{\infty} \frac{f_d(t_{id}^{clone}|u,x_i)f_s(u|x_i)}{Pr(\tau_{contract}<\tau_{i,birth}+T_{id}^{seller}<\tau_{end}|X_i=x_i)}du \\$
<br>
<br>
$= \displaystyle \frac{\lambda_d \phi_d(x_i)e^{-\phi_d(x_i)I_d(t_{id}^{clone})}e^{-\phi_s(x_i)I_s(t_{id}^{clone})}}{\int \left[e^{-\phi_d(x_i)I_d(\tau_{contract}-\tau_{i,birth},t_s)}-e^{-\phi_d(x_i)I_d(\tau_{end}-\tau_{i,birth},t_s)}\right] \lambda_s \phi_s(x_i) e^{-\phi_s(x_i)I_s(t_s)} dt_s}.$ <br>
<br>
<br>
Comme ci-dessus on spécifie $\phi_j(x_i)=exp(x_i'\beta_j)$, $j=d,s$, où $x_i'\beta_j=x_{i1}\beta_{j1}+x_{i2}\beta_{j2}+ ...+x_{iK}\beta_{jK}$, avec $K$ le nombre de variables explicatives.

Type of solver in scikit.spicy.minimize : [Have a look](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html) <br>

- ‘Nelder-Mead’  <br>

- ‘Powell’ 

- ‘CG’ 

- ‘BFGS’ 

- ‘Newton-CG’ 

- ‘L-BFGS-B’ 

- ‘TNC’ 

- ‘COBYLA’ 

- ‘SLSQP’ 

- ‘trust-constr’

- ‘dogleg’ 

- ‘trust-ncg’ 

- ‘trust-exact’ 

- ‘trust-krylov’ 

In [ ]:
# Définir la fonction de vraisemblance (à maximiser) 
def likelihood(params):
    # Calculer la vraisemblance en fonction des paramètres
    #
    #
    # Retourner le négatif de la vraisemblance car minimize minimise la fonction
    return -likelihood_value

# Définir la contribution Li_seller 
def L_seller():


    return L_seller # un vecteur de taille N

In [ ]:
# Initialisation des paramètres
initial_params = np.array([initial_value_1, initial_value_2, ...])

# Utiliser l'algorithme de minimisation de SciPy pour maximiser la fonction de vraisemblance
result = minimize(likelihood, initial_params, method='BFGS')  # Vous pouvez choisir d'autres méthodes aussi

# Les paramètres optimisés se trouvent dans result.x
optimized_params = result.x